In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse 
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
import dask.dataframe as dd
from dask.dataframe import from_pandas
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from prettytable import PrettyTable

In [8]:
df=pd.read_csv('/content/drive/MyDrive/feature_extracted')

In [9]:
df_scraped=pd.read_csv('/content/drive/MyDrive/scraped_df')

In [10]:
columns=['length',
       'no_of_words', 'avg_word_length', 'caps', 'excl', 'sentence_count',
       'avg_sentence_length', 'sentiment', 'subjectivity', 'noun_count',
       'verb_count', 'adj_count', 'adv_count', 'pron_count']

In [11]:
identity_columns = ['asian',
       'atheist', 'bisexual', 'black', 'buddhist', 'christian', 'female',
       'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity',
       'other_religion', 'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']

In [12]:
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, 1, 0)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

df = convert_dataframe_to_bool(df)

In [13]:
df_non_toxic=df[df['target']==0].sample(frac=0.1)
df_toxic=df[df['target']==1]

In [14]:
df_new=pd.concat([df_toxic,df_non_toxic],axis=0)

In [15]:
Y=df_new['target']
X=df_new.drop(['target'],axis=1)

In [16]:
len(X)

307990

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42,stratify=Y, shuffle=True)

In [18]:
X_train=X_train[['clean_text','length',
       'no_of_words', 'avg_word_length', 'caps', 'excl', 'sentence_count',
       'avg_sentence_length', 'sentiment', 'subjectivity', 'noun_count',
       'verb_count', 'adj_count', 'adv_count', 'pron_count']]

In [19]:
X_scraped=df_scraped.drop(['Unnamed: 0', 'text', 'target'],axis=1)
Y_scraped=df_scraped['target']

In [20]:
X_train_scrapped=pd.concat([X_train,X_scraped],axis=0)

In [21]:
y_train_scraped=pd.concat([y_train,Y_scraped],axis=0)

In [22]:
vectorizer = TfidfVectorizer()
tfidf_vect = vectorizer.fit(X_train['clean_text'])

In [23]:
vectorizer1 = TfidfVectorizer()
vectorizer1.fit(X_train_scrapped['clean_text'])

TfidfVectorizer()

In [24]:
import pickle
with open('/content/drive/MyDrive/glove.840B.300d.pkl', 'rb') as fp:
    glove = pickle.load(fp)
glove_words=glove.keys()

In [19]:
dictionary=dict(zip(vectorizer.get_feature_names(),list(vectorizer.idf_)))
word_list=set(vectorizer.get_feature_names())
X_train_w2v=[]
count={0}
for sentence in tqdm(X_train['clean_text']):
    vector=np.zeros(300)    
    tfidf_weight=0    
    l=len(sentence.split())
    for word in sentence.split():
        if word in glove_words and word in word_list:
            count.add(word)
            vec=glove[word]               
            tfidf=dictionary[word]*(sentence.count(word)/l)            
            vector+=(vec*tfidf)            
            tfidf_weight+=tfidf
    if tfidf_weight !=0:
        vector/=tfidf_weight
    X_train_w2v.append(vector) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 200193/200193 [00:49<00:00, 4055.99it/s]


In [ ]:
dictionary=dict(zip(vectorizer.get_feature_names(),list(vectorizer.idf_)))
word_list=set(vectorizer.get_feature_names())
X_test_w2v=[]
count={0}
for sentence in tqdm(X_test['clean_text']):
    vector=np.zeros(300)    
    tfidf_weight=0    
    l=len(sentence.split())
    for word in sentence.split():
        if word in glove_words and word in word_list:
            count.add(word)
            vec=glove[word]               
            tfidf=dictionary[word]*(sentence.count(word)/l)            
            vector+=(vec*tfidf)            
            tfidf_weight+=tfidf
    if tfidf_weight !=0:
        vector/=tfidf_weight
    X_test_w2v.append(vector)

In [40]:
parameters={'max_depth':[4,5,6,7,8]}
clf = HistGradientBoostingClassifier()
clf = GridSearchCV(clf, parameters,cv=5,scoring='roc_auc')
clf.fit(X_train_w2v,y_train)

GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(),
             param_grid={'max_depth': [4, 5, 6, 7, 8]}, scoring='roc_auc')

In [41]:
clf.best_estimator_

HistGradientBoostingClassifier(max_depth=7)

In [43]:
clf = HistGradientBoostingClassifier(max_depth=7)
clf.fit(X_train_w2v,y_train)
Y_pred_tr=clf.predict_proba(X_train_w2v)

In [47]:
Y_pred_te=clf.predict_proba(X_test_w2v)

In [45]:
roc_auc_score(y_train, Y_pred_tr[:,1])

0.8742061001244199

In [48]:
roc_auc_score(y_test, Y_pred_te[:,1])

0.8551697983387323

In [26]:
Y_pred_te=NB.predict_proba(X_test_w2v)

In [49]:
X_te_bias=X_test[identity_columns]
X_te_bias['y_pred']=Y_pred_te[:,1]
X_te_bias['y']=y_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
identity_columns1=['latino','transgender','asian','psychiatric_or_mental_illness','jewish','homosexual_gay_or_lesbian',
                  'black','muslim','white','christian','male','female']

In [43]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup):
    subgroup_examples = df[df[subgroup]==1]
    return compute_auc(subgroup_examples['y'], subgroup_examples['y_pred'])

def compute_bpsn_auc(df, subgroup):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup]==1]
    subgroup_negative_examples = subgroup_negative_examples[subgroup_negative_examples['y']==0]
    non_subgroup_positive_examples = df[df[subgroup]==0]
    non_subgroup_positive_examples = non_subgroup_positive_examples[non_subgroup_positive_examples['y']==1]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples['y'], examples['y_pred'])

def compute_bnsp_auc(df, subgroup):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup]==1]
    subgroup_positive_examples = subgroup_positive_examples[subgroup_positive_examples['y']==1]
    non_subgroup_negative_examples = df[df[subgroup]==0]
    non_subgroup_negative_examples = non_subgroup_negative_examples[non_subgroup_negative_examples['y']==0]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples['y'], examples['y_pred'])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {'subgroup': subgroup,'subgroup_size': len(dataset[dataset[subgroup]==1])}
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_size', ascending=True)

In [53]:
bias_metrics_df = compute_bias_metrics_for_model(X_te_bias, identity_columns1)
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
0,latino,196,0.809229,0.732885,0.903439
1,transgender,264,0.730841,0.755053,0.843373
2,asian,330,0.812011,0.778725,0.882565
3,psychiatric_or_mental_illness,487,0.784810,0.800037,0.843186
4,jewish,669,0.765851,0.694391,0.898616
5,homosexual_gay_or_lesbian,1353,0.755046,0.711706,0.887529
6,black,2072,0.729678,0.653142,0.902898
7,muslim,2225,0.751617,0.659452,0.910314
9,christian,2579,0.825745,0.837338,0.845654
8,white,3075,0.731784,0.634261,0.912499


In [44]:
def score(df,overall_auc):
    l=len(df)
    x=df.drop(['subgroup','subgroup_size'],axis=1)
    x=x.apply((lambda x: x**(-5)),axis=1)
    s=x.sum(axis=0)
    s=s.apply(lambda x: (x/l)**(1/(-5)))
    s=s.sum(axis=0)
    score=0.25*(s+overall_auc)
    return score    

In [56]:
bias_score=score(bias_metrics_df,0.855)
bias_score

0.8054831489463242

In [32]:
dictionary=dict(zip(vectorizer1.get_feature_names(),list(vectorizer1.idf_)))
word_list=set(vectorizer1.get_feature_names())
X_train_scraped_w2v=[]
count={0}
for sentence in tqdm(X_train_scrapped['clean_text']):
    vector=np.zeros(300)    
    tfidf_weight=0    
    l=len(sentence.split())
    for word in sentence.split():
        if word in glove_words and word in word_list:
            count.add(word)
            vec=glove[word]               
            tfidf=dictionary[word]*(sentence.count(word)/l)            
            vector+=(vec*tfidf)            
            tfidf_weight+=tfidf
    if tfidf_weight !=0:
        vector/=tfidf_weight
    X_train_scraped_w2v.append(vector)  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 214331/214331 [01:13<00:00, 2933.74it/s]


In [33]:
dictionary=dict(zip(vectorizer1.get_feature_names(),list(vectorizer1.idf_)))
word_list=set(vectorizer1.get_feature_names())
X_test_scraped_w2v=[]
count={0}
for sentence in tqdm(X_test['clean_text']):
    vector=np.zeros(300)    
    tfidf_weight=0    
    l=len(sentence.split())
    for word in sentence.split():
        if word in glove_words and word in word_list:
            count.add(word)
            vec=glove[word]               
            tfidf=dictionary[word]*(sentence.count(word)/l)            
            vector+=(vec*tfidf)            
            tfidf_weight+=tfidf
    if tfidf_weight !=0:
        vector/=tfidf_weight
    X_test_scraped_w2v.append(vector) 

100%|██████████| 107797/107797 [00:33<00:00, 3218.76it/s]


In [29]:
parameters={'max_depth':[4,5,6,7,8]}
clf = HistGradientBoostingClassifier()
clf = GridSearchCV(clf, parameters,cv=5,scoring='roc_auc')
clf.fit(X_train_scraped_w2v,y_train_scraped)

GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(),
             param_grid={'max_depth': [4, 5, 6, 7, 8]}, scoring='roc_auc')

In [30]:
clf.best_estimator_

HistGradientBoostingClassifier(max_depth=8)

In [36]:
clf = HistGradientBoostingClassifier(max_depth=8)
clf.fit(X_train_scraped_w2v,y_train_scraped)
Y_pred_tr=clf.predict_proba(X_train_scraped_w2v)

In [38]:
Y_pred_te=clf.predict_proba(X_test_scraped_w2v)

In [39]:
roc_auc_score(y_train_scraped, Y_pred_tr[:,1])

0.8827457462430206

In [40]:
roc_auc_score(y_test, Y_pred_te[:,1])

0.8520936277352503

In [41]:
X_te_bias=X_test[identity_columns]
X_te_bias['y_pred']=Y_pred_te[:,1]
X_te_bias['y']=y_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
bias_metrics_df = compute_bias_metrics_for_model(X_te_bias, identity_columns1)
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
0,latino,198,0.759318,0.710047,0.890941
1,transgender,254,0.735246,0.814572,0.783749
2,asian,332,0.795904,0.779590,0.868160
3,psychiatric_or_mental_illness,502,0.756110,0.801848,0.815547
4,jewish,647,0.747204,0.720878,0.874166
5,homosexual_gay_or_lesbian,1339,0.715342,0.733147,0.845361
6,black,2034,0.713714,0.647084,0.894949
7,muslim,2221,0.742333,0.677204,0.895699
9,christian,2554,0.822677,0.850253,0.825847
8,white,3055,0.719175,0.632638,0.903999


In [49]:
bias_score=score(bias_metrics_df,0.852)
bias_score

0.7969099640313553

In [52]:
x = PrettyTable()
x.field_names = ['Feature',"Train AUC", "Test AUC", "Bias_Score"]
x.add_rows(
    [
        ["W2v", 0.8827, 0.8551, 0.8054],
        ["W2v with scraped data",  0.8742, 0.8520, 0.7969]          
    ]
)

In [53]:
print(x)

+-----------------------+-----------+----------+------------+
|        Feature        | Train AUC | Test AUC | Bias_Score |
+-----------------------+-----------+----------+------------+
|          W2v          |   0.8827  |  0.8551  |   0.8054   |
| W2v with scraped data |   0.8742  |  0.852   |   0.7969   |
+-----------------------+-----------+----------+------------+
